In [1]:
# import packages
from langchain_community.llms import Ollama

## Loading our llm model - Note the ollama server should be running

In [2]:
# initialize our llm model
llm = Ollama(model='llama3')

In [3]:
llm.invoke('how can langsmith help with testing')

"A great question!\n\nLinguamatic (not Langsmith, but I assume it's a similar concept) is an AI-powered language generation platform that can indeed be leveraged to facilitate testing in various ways. Here are some examples:\n\n1. **Automated Testing**: Linguamatic can generate test cases, input data, or expected output based on natural language descriptions. This can save time and effort in creating test scenarios.\n2. **Test Data Generation**: The platform can produce a wide range of realistic test data (e.g., text, numbers, dates) to simulate various user inputs, allowing you to cover more testing scenarios.\n3. **Error Detection**: Linguamatic's natural language processing capabilities can identify potential errors or inconsistencies in your code, helping you catch bugs early on.\n4. **Testing for Language-Specific Features**: If your application has features that rely heavily on language understanding (e.g., text analysis, sentiment analysis), Linguamatic can assist in testing the

## Creating and Using a Chat prompt template

In [3]:
# laoding a template to help refine our model response
from langchain_core.prompts import ChatPromptTemplate

In [8]:
prompt = ChatPromptTemplate(
    [
        ("system", "you are a renowned technical writer"),
        ("user", "{input}")
    ]
)

# chaining our prompt with the llm
chain = prompt | llm

# invoking the llm
response = chain.invoke({"input": "how can langsmith help with testing"})

In [9]:
print(response
      )

As a technical writer, I'm delighted to share some insights on how Langsmith can assist with testing.

Langsmith is an excellent tool for testing machine learning models. Here are some ways it can help:

1. **Automated Testing**: Langsmith provides automated testing capabilities, which means you can run tests repeatedly without manual intervention. This helps ensure that your model's performance remains consistent across different inputs and scenarios.
2. **Model Interpretability**: Langsmith allows you to visualize the internal workings of your machine learning models. By analyzing feature importance, partial dependence plots, and SHAP values, you can gain insights into how the model is making predictions. This helps identify potential biases or errors in the model's behavior.
3. **Edge Case Detection**: Langsmith's ability to generate synthetic data allows you to test edge cases that might not be easily replicable through manual testing. By feeding these generated inputs into your mo

## Utilizing a output parser to refine the response of our llm model 

In [10]:
# adding a output parser to convert our chat message to a string
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

# chain the output parser to the chat model
chain = prompt | llm | output_parser

#call the chain invoke method to get a response
response = chain.invoke({"input": "how can langsmith help with testing"})

"As a technical writer, I'm excited to share the ways Langsmith can facilitate effective testing for your project.\n\n**Automated Testing**: Langsmith's ability to generate natural language tests (NLTs) can significantly reduce the effort required to create automated test cases. By leveraging Langsmith's AI-powered testing capabilities, you can:\n\n1. **Quickly generate test data**: Create realistic test data that accurately reflects real-world scenarios.\n2. **Automate tedious testing tasks**: Langsmith can automatically execute repetitive tests, freeing up your team to focus on more complex and creative testing efforts.\n\n**Test Case Generation**: Langsmith's AI-powered language processing capabilities allow it to analyze requirements documents, user stories, or acceptance criteria to generate test cases. This can help:\n\n1. **Ensure comprehensive coverage**: Generate a set of test cases that thoroughly cover the required scenarios.\n2. **Reduce manual testing effort**: Automate th

In [15]:
response.rsplit(sep='\n\n')

["As a technical writer, I'm excited to share the ways Langsmith can facilitate effective testing for your project.",
 "**Automated Testing**: Langsmith's ability to generate natural language tests (NLTs) can significantly reduce the effort required to create automated test cases. By leveraging Langsmith's AI-powered testing capabilities, you can:",
 '1. **Quickly generate test data**: Create realistic test data that accurately reflects real-world scenarios.\n2. **Automate tedious testing tasks**: Langsmith can automatically execute repetitive tests, freeing up your team to focus on more complex and creative testing efforts.',
 "**Test Case Generation**: Langsmith's AI-powered language processing capabilities allow it to analyze requirements documents, user stories, or acceptance criteria to generate test cases. This can help:",
 '1. **Ensure comprehensive coverage**: Generate a set of test cases that thoroughly cover the required scenarios.\n2. **Reduce manual testing effort**: Automa

In [16]:
### Arguably it seems without the outputparser the initial response outlook was far more better

## Creating a Retriever (a vectorstore) that would serve as our body of Knoweledge

In [4]:
from langchain_community.document_loaders import WebBaseLoader

In [20]:
# NB: Setup your user-agent environment variable to use the webbaseloader 
# Try out the command - curl -s -I https://www.google.com | grep "User-Agent"
# This will return the user-agent used by curl to access the website. Note the default browser in question is what is used.
# Another alternative could be to use chrome developer tool to obtain your user-agent environment variable

In [5]:
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")
docs = loader.load()

In [6]:
# Indexing our docs into a vectorstore
# first create our embedding object

from langchain_community.embeddings import OllamaEmbeddings
embeddings  = OllamaEmbeddings(model="llama3")

In [7]:
# Now, we can use this embedding model to ingest documents into a vectorstore. 
# We will use a simple local vectorstore, FAISS, for now
# For FAISS, install the library faiss-cpu
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


# Building our index

# instantiate our text splitter
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [8]:
# Now that we have this data indexed in a vectorstore, we will create a retrieval chain. This chain will take an incoming question, 
# look up relevant documents, then pass those documents along with the original question into an LLM and ask it to answer the original question.


# Setting up a chain that takes a question and the retrived document and generates an answer
from langchain.chains.combine_documents.stuff import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:
<context>
{context}
</context>
                                 
Question: {input}""")


document_chain = create_stuff_documents_chain(prompt=prompt, llm=llm)

In [9]:
# creating our retrieval chain
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)


# we call the invoke method on the retrieval chain, passing in a query
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})

In [33]:
print(response['answer'])

According to the provided context, LangSmith helps with testing in the following ways:

1. **Test cases**: LangSmith allows developers to create datasets, which are collections of inputs and reference outputs, and use these to run tests on their LLM applications.
2. **Comparison view**: LangSmith provides a user-friendly comparison view for test runs to track and diagnose regressions in test scores across multiple revisions of your application.
3. **Automated evaluations**: Langsmith also makes it easy to run custom evaluations (both LLM and heuristic based) to score test results.

Overall, LangSmith helps developers create, manage, and evaluate test cases to ensure their LLM applications are performing correctly and consistently.


# Expanding our retrieval chain into a conversational retrieval system

In [10]:
# importing functions to help create and handle chat history
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder


# updating our chat prompt template
prompt = ChatPromptTemplate([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])

retriever_chain_with_history = create_history_aware_retriever(llm, retriever, prompt)

In [11]:
#  testing out our retriever chain with history
# creating an instance of a follow up question
from langchain_core.messages import AIMessage, HumanMessage

In [37]:
chat_history = [HumanMessage(content= "Can LangSmith help test my LLM applications?"), AIMessage(content = "Yes !")]
retriever_chain_with_history.invoke(
    {
        "chat_history": chat_history,
        "input": "Tell me how?"
    }
)

[Document(metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.', 'language': 'en'}, page_content='meaning that they involve a series of interactions between the user and the application. LangSmith provides a threads view that groups traces from a single conversation together, making it easier to track the performance of and annotate your application across multiple turns.Was this page helpful?You can leave detailed feedback on GitHub.PreviousQuick StartNextOverviewPrototypingBeta TestingProductionCommunityDiscordTwitterGitHubDocs Cod

In [12]:
# Now that we have this new retriever, we can create a new chain to continue 
# the conversation with these retrieved documents in mind.
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain_update = create_retrieval_chain(retriever_chain_with_history, document_chain)

In [39]:
# testing end-2-end
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes !")]
retrieval_chain_update.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes !')],
 'input': 'Tell me how',
 'context': [Document(metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.', 'language': 'en'}, page_content='meaning that they involve a series of interactions between the user and the application. LangSmith provides a threads view that groups traces from a single conversation together, making it easier to track the performance of and annotate your application across multiple turns.Was this 

In [41]:
response = retrieval_chain_update.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

print(response['answer'])

According to the context, LangSmith can help test your LLM (Large Language Model) applications in several ways:

1. **Prototyping**: LangSmith allows rapid experimentation between prompts, model types, retrieval strategy, and other parameters. You can quickly test out different prompts and models in the playground environment.

2. **Debugging**: When things go wrong, Langsmith gives clear visibility and debugging information at each step of an LLM sequence, making it easier to identify and root-cause issues.

3. **Initial Test Set**: You can create datasets, which are collections of inputs and reference outputs, and use these to run tests on your LLM applications. LangSmith also makes it easy to run custom evaluations (both LLM and heuristic-based) to score test results.

4. **Comparison View**: When prototyping different versions of your applications and making changes, Langsmith provides a user-friendly comparison view for test runs to track and diagnose regressions in test scores ac

AGENTS

Our agent needs two important tools:
 - a retriever 
 - a search tool

In [13]:
# Setting up a retriever tool
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    name="retriever",
    description="Searches and returns information about the most relevant document based on the user's query.",
)

In [15]:
# setting up our search tool
from langchain_community.tools.tavily_search import TavilySearchResults

# to load our api-key for tavily from our environment variables
from dotenv import load_dotenv

load_dotenv()

search = TavilySearchResults()

In [16]:
# setting up the complete tooling
tool = [retriever_tool, search]

In [17]:
# Installing some key packages for the project
# the following can be installed - langchainhub, langchain-ollama

from langchain_ollama import ChatOllama
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

In [18]:
prompt = hub.pull("hwchase17/openai-functions-agent")
llm = ChatOllama(
    model = "llama3",  
    temperature = 0.8,  
    num_predict = 256, 
)
agent = create_openai_functions_agent(
    llm, 
    tool, 
    prompt
)
agent_executor = AgentExecutor(agent=agent, tools=tool, verbose=True)

/Users/eugene/Personal_Projects/ML_Project/llm_app_question_and_answer/.venv/lib/python3.10/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


In [19]:
agent_executor.invoke({"input": "how can langsmith help with testing?"})



> Entering new AgentExecutor chain...
Linguine is an AI-powered language understanding platform that can significantly aid in testing various aspects of your product or service. Here are some ways Linguine can assist:

1. **Natural Language Processing (NLP) Testing**: Linguine can be used to test the NLP capabilities of your application, such as text classification, sentiment analysis, entity recognition, and more. You can use Linguine's API to feed your application a wide range of texts with different characteristics, styles, and nuances.

2. **Content Generation**: Linguine can generate large amounts of text data in various formats, languages, and styles. This can be useful for testing content-based applications, such as AI-powered chatbots or recommendation systems.

3. **Edge Case Testing**: Linguine's ability to understand language patterns and variations enables you to test edge cases that might be difficult to identify manually. For instance, you can use Linguine to generate t

{'input': 'how can langsmith help with testing?',
 'output': "Linguine is an AI-powered language understanding platform that can significantly aid in testing various aspects of your product or service. Here are some ways Linguine can assist:\n\n1. **Natural Language Processing (NLP) Testing**: Linguine can be used to test the NLP capabilities of your application, such as text classification, sentiment analysis, entity recognition, and more. You can use Linguine's API to feed your application a wide range of texts with different characteristics, styles, and nuances.\n\n2. **Content Generation**: Linguine can generate large amounts of text data in various formats, languages, and styles. This can be useful for testing content-based applications, such as AI-powered chatbots or recommendation systems.\n\n3. **Edge Case Testing**: Linguine's ability to understand language patterns and variations enables you to test edge cases that might be difficult to identify manually. For instance, you ca